## Set up

In [ ]:
import os

# Find the latest version of spark 2.0 from http://www-us.apache.org/dist/spark/ and update the line below if necessary
spark_version = 'spark-2.4.7'
os.environ['SPARK_VERSION'] = spark_version

# Install dependencies: Spark, hadoop, Java, and Findspark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
# Set environment path in order to run PySpark in Google Colab
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

In [ ]:
# Create a local Spark session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

## Import data

In [ ]:
# Upload heroes_information.csv and super_hero_powers.csv files
from google.colab import files
files.upload()

In [ ]:
# Import data from both CSV files into DataFrames
heroes = spark.read.load(
    "heroes_information.csv",
    format="csv",
    inferSchema="true",
    header="true"
)

superpowers = spark.read.load(
    "super_hero_powers.csv",
    format="csv",
    inferSchema="true",
    header="true"
)

In [ ]:
# Create views from the DataFrames so that we can query our data in SQL
heroes.createOrReplaceTempView("heroes")
superpowers.createOrReplaceTempView("superpowers")

## Write SQL queries

In [ ]:
# Write a SQL query to return the name, eye color, and alignment of heroes that have an alignment of 'bad' or 'neutral' and the power of invisibility
sqlSneakyInvisibles = spark.sql("""
SELECT heroes.name, heroes.`Eye color`, heroes.alignment
FROM heroes
JOIN superpowers ON heroes.name = superpowers.hero_names
WHERE heroes.alignment IN ('bad', 'neutral')
  AND superpowers.invisibility = True
""")

sqlSneakyInvisibles.show()

In [ ]:
# Write a SQL query to return the top ten publishers by count of female heroes with the power of super strength
sqlFemalesByPublisher = spark.sql("""
SELECT publisher, count(1)
FROM heroes
JOIN superpowers ON heroes.name = superpowers.hero_names
WHERE heroes.gender = 'Female'
  AND superpowers.`Super strength` = True
GROUP BY heroes.publisher
ORDER BY count(1) DESC
LIMIT 10
""")

sqlFemalesByPublisher.show()

In [ ]:
# Write a SQL query to return all of the heroes that exist in the heroes table but not the superpowers table
sqlHeroesWithoutPowers = spark.sql("""
SELECT heroes.name
FROM heroes
LEFT JOIN superpowers ON heroes.name = superpowers.hero_names
WHERE superpowers.hero_names IS NULL
""")

sqlHeroesWithoutPowers.show()